In [1]:
from IPython.display import HTML
from dataset import TUBerlinDataset
from prepare_data import convert_and_quantize_svg

labels = ["backpack"]
dataset = TUBerlinDataset(labels, download=True)

originals = ""
quantized = ""

for svg_content in dataset[0:3]:
    originals += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original, length: {len(svg_content)}</b><br>{svg_content}</div>'
    output = convert_and_quantize_svg(svg_content, bins=256)
    quantized += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Quantized, length: {len(output)}</b><br>{output}</div>'

display(HTML(originals))
display(HTML(quantized))

Loading TUBerlin files: 100%|██████████| 1/1 [00:00<00:00,  5.99it/s]


In [2]:
from IPython.display import HTML
from dataset import QuickDrawDataset
from prepare_data import convert_and_quantize_svg

labels = ["bulldozer"]
dataset = QuickDrawDataset(labels, download=True)

originals = ""
quantized = ""

for svg_content in dataset[0:3]:
    originals += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original, length: {len(svg_content)}</b><br>{svg_content}</div>'
    output = convert_and_quantize_svg(svg_content, bins=128)
    quantized += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Quantized, length: {len(output)}</b><br>{output}</div>'

display(HTML(originals))
display(HTML(quantized))

Loading QuickDraw files: 100%|██████████| 1/1 [00:04<00:00,  4.76s/it]


## Point-based representation

In [3]:
from IPython.display import HTML
from utils import svg_to_tensor, tensor_to_svg, svg_to_tensor_quantized

original_svg = """<svg viewBox="0 0 256 256"><g stroke-width="0.8">
<path d="M 26 22 L 32 50 L 33 84" stroke="black" fill="none"/>
<path d="M 24 24 L 56 16 L 114 7 L 113 75 L 111 82 L 105 86" stroke="black" fill="none"/>
<path d="M 8 101 L 26 96 L 130 84" stroke="black" fill="none"/>
<path d="M 7 103 L 6 153 L 11 156 L 24 157 L 59 150 L 127 147" stroke="black" fill="none"/>
<path d="M 138 86 L 132 142 L 134 148" stroke="black" fill="none"/>
<path d="M 133 85 L 133 78 L 157 18 L 173 0 L 189 24 L 203 57 L 207 91 L 196 106 L 195 126 L 203 134 L 214 138 L 229 139 L 237 136 L 247 128 L 240 105 L 226 89 L 209 87 L 204 92" stroke="black" fill="none"/>
<path d="M 202 140 L 209 147 L 211 146 L 215 127 L 221 141 L 225 140 L 232 130 L 238 137 L 247 124 L 255 121" stroke="black" fill="none"/>
<path d="M 201 141 L 201 141" stroke="black" fill="none"/>
<path d="M 200 128 L 204 137" stroke="black" fill="none"/>
<path d="M 2 163 L 38 150 L 155 137 L 163 139 L 166 149 L 166 156 L 158 166 L 148 173 L 121 180 L 41 191 L 23 191 L 7 185 L 1 176 L 1 160 L 14 152" stroke="black" fill="none"/>
</g></svg>"""
tensor = svg_to_tensor(original_svg)
reconstructed_svg = tensor_to_svg(tensor)

print("svg -> tensor -> svg")
svgs_inline = f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original</b><br>{original_svg}</div>'
svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Reconstructed</b><br>{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# Note: quantization is faster when converting directly from svg to quantized tensor
print("svg -> quantized tensor -> svg")
quantization_levels = [16, 32, 64, 128, 256]
svgs_inline = ""
for bins in quantization_levels:
    quantized_tensor = svg_to_tensor_quantized(original_svg, bins=bins)
    reconstructed_svg = tensor_to_svg(quantized_tensor, size=bins)
    svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;">{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# print(tensor.shape)
# print(tensor)
# print(tensor.nbytes)

# If quantized to 128 bins, the tensor can be stored as int8

svg -> tensor -> svg


svg -> quantized tensor -> svg


## Stroke-based representation

In [4]:
from IPython.display import HTML
from utils import (
    svg_strokes_to_tensor,
    svg_strokes_to_tensor_quantized,
    tensor_to_svg_strokes,
)
import torch

original_svg = """<svg viewBox="0 0 256 256"><g stroke-width="0.8">
<path d="M 26 22 L 32 50 L 33 84" stroke="black" fill="none"/>
<path d="M 24 24 L 56 16 L 114 7 L 113 75 L 111 82 L 105 86" stroke="black" fill="none"/>
<path d="M 8 101 L 26 96 L 130 84" stroke="black" fill="none"/>
<path d="M 7 103 L 6 153 L 11 156 L 24 157 L 59 150 L 127 147" stroke="black" fill="none"/>
<path d="M 138 86 L 132 142 L 134 148" stroke="black" fill="none"/>
<path d="M 133 85 L 133 78 L 157 18 L 173 0 L 189 24 L 203 57 L 207 91 L 196 106 L 195 126 L 203 134 L 214 138 L 229 139 L 237 136 L 247 128 L 240 105 L 226 89 L 209 87 L 204 92" stroke="black" fill="none"/>
<path d="M 202 140 L 209 147 L 211 146 L 215 127 L 221 141 L 225 140 L 232 130 L 238 137 L 247 124 L 255 121" stroke="black" fill="none"/>
<path d="M 201 141 L 201 141" stroke="black" fill="none"/>
<path d="M 200 128 L 204 137" stroke="black" fill="none"/>
<path d="M 2 163 L 38 150 L 155 137 L 163 139 L 166 149 L 166 156 L 158 166 L 148 173 L 121 180 L 41 191 L 23 191 L 7 185 L 1 176 L 1 160 L 14 152" stroke="black" fill="none"/>
</g></svg>"""
tensor = svg_strokes_to_tensor(original_svg)
tensor = tensor.to(torch.int16)
reconstructed_svg = tensor_to_svg_strokes(tensor)

print("svg -> tensor -> svg")
svgs_inline = f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original</b><br>{original_svg}</div>'
svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Reconstructed</b><br>{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# Note: quantization is faster when converting directly from svg to quantized tensor
print("svg -> quantized tensor -> svg")
quantization_levels = [16, 32, 64, 128, 256]
svgs_inline = ""
for bins in quantization_levels:
    quantized_tensor = svg_strokes_to_tensor_quantized(original_svg, bins=bins)
    reconstructed_svg = tensor_to_svg_strokes(quantized_tensor, size=bins)
    svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;">{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# print(tensor.shape)
# print(tensor)
# print(tensor.nbytes)

svg -> tensor -> svg


svg -> quantized tensor -> svg
